# Data Preperation

## Input:
- main_data_folder path from this file which will hold all data outside of git. This path folder is in the same folder that holds the GitHub folder of Diabetic-Retinopathy-Detection

##### Inside of main_data_folder there must be placed:
- 1) CSV of Targets for each image
- 2) train folder with all unzipped image data (38.1 GB total from all 5 train zipped files)

## Output:
- 1) downsized_data folder which includes 5 class folder that will be used for both training and future data augmentation. The images are 500 x 500 x 3 pixels each. The validate and test images are not in the folder.
- 2) data folder
- 3) Inside of the data folder there is a train, val, and test folder. The train folder is empty.
- 4) The val & test folder each have 71 randomely selected images for each class. These folders does not contain images in the downsized_data folder.

### Next Steps:
- Augment data in the downsized_data and put all new training data in the train folder

In [1]:
import numpy as np
import cv2
import os
import shutil
from distutils.dir_util import copy_tree

# https://stackoverflow.com/questions/82831/how-do-i-check-whether-a-file-exists-without-exceptions/44661513
from pathlib import Path as path_check

# Input
- Main Folder of Data


In [2]:
main_data_folder = '../SML_Project_Data'

# Analysis of a Single Image

In [3]:
# Load an color image in color
img = cv2.imread( main_data_folder + '/train/10_left.jpeg',1)
print(img.shape)
img

(3168, 4752, 3)


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

### Create PNG of Image Object

In [4]:
cv2.imwrite('10_left_new.jpeg',img) # Confirmed manually the exported image is the same number of bytes as the original image

True

### Inspect Image Object

In [5]:
img.shape

(3168, 4752, 3)

In [6]:
img[1500][2300][1]

146

In [7]:
np.max(img)

255

In [8]:
np.min(img)

0

In [9]:
print('Original Size', img.shape)

print("%d bytes in original image" % (np.prod(img.shape)))

print('MB in the original array image is', img.size * img.itemsize / 1000000, 'versus the original image JPEG has 1.5 MB when the JPEG has comression versus a tensor') # 1 million bytes in a MB

resized = cv2.resize(img, (500, 500), interpolation = cv2.INTER_AREA)
print('\nResized Size', resized.shape)
cv2.imwrite('10_left_reshaped_new.jpeg',resized) # Goes from 1.5 MB to 3 KB which redices the size by a factor 1 K

print("%d bytes in compressed image" % (resized.size * resized.itemsize))

print('New pixel size is', np.prod(resized.shape))
print('Factor of reduction is by', np.prod(img.shape) / np.prod(resized.shape))

print('To get 100 K or so images in total, this would be around 5 times the original data set or a factor of original set of', np.prod(img.shape) / (np.prod(resized.shape) * 5 ) )
print('Results in GB of augmented data of', 50 / (np.prod(img.shape) / (np.prod(resized.shape) * 5 )))

Original Size (3168, 4752, 3)
45163008 bytes in original image
MB in the original array image is 45.163008 versus the original image JPEG has 1.5 MB when the JPEG has comression versus a tensor

Resized Size (500, 500, 3)
750000 bytes in compressed image
New pixel size is 750000
Factor of reduction is by 60.217344
To get 100 K or so images in total, this would be around 5 times the original data set or a factor of original set of 12.0434688
Results in GB of augmented data of 4.1516278100874064


# Downscale Images to 500 x 500 x 3 and Export

In [10]:
from pathlib import Path
import pandas as pd

In [11]:
train_target = pd.read_csv(main_data_folder + '/trainLabels.csv', delimiter=',')
print(len(train_target))
train_target.head()

35126


,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


### Count Images

In [13]:
total_images = 0
for index, row in train_target.iterrows():
    split_image = row['image'].split('_')
    image_num = split_image[0]
    side = split_image[1]
    
    image_name = str(image_num) + '_' + side + '.jpeg'

    train_image_path = main_data_folder + '/train/' + image_name

    my_file = Path(train_image_path)

    try:
        my_abs_path = my_file.resolve(strict=True)
    except FileNotFoundError:
        pass
    else:
        total_images = total_images + 1
total_images

35126

# Create Folders to Store All Downsized Data

In [14]:
def create_folder(path, replace = False):
    if replace:
        print('Attempting to delete folder of', path)
        try:
            shutil.rmtree(path)
        except FileNotFoundError:
            print('Folder already does not exist')
        else:
            pass
    
    print('Looking to create folders of', path)
    try:
        os.mkdir(path)
        print('')
    except FileExistsError:
        print('Folder Already Exists in ' + path + '\n')
    else:
        pass

# Create folders for each
downsized_data_path = main_data_folder + '/downsized_data'

create_folder(downsized_data_path)

Looking to create folders of ../SML_Project_Data/downsized_data
Folder Already Exists in ../SML_Project_Data/downsized_data



In [15]:
for disease in (0, 1, 2, 3, 4):
    path = downsized_data_path + '/resized_' + str(disease)
    create_folder(path)

Looking to create folders of ../SML_Project_Data/downsized_data/resized_0
Folder Already Exists in ../SML_Project_Data/downsized_data/resized_0

Looking to create folders of ../SML_Project_Data/downsized_data/resized_1
Folder Already Exists in ../SML_Project_Data/downsized_data/resized_1

Looking to create folders of ../SML_Project_Data/downsized_data/resized_2
Folder Already Exists in ../SML_Project_Data/downsized_data/resized_2

Looking to create folders of ../SML_Project_Data/downsized_data/resized_3
Folder Already Exists in ../SML_Project_Data/downsized_data/resized_3

Looking to create folders of ../SML_Project_Data/downsized_data/resized_4
Folder Already Exists in ../SML_Project_Data/downsized_data/resized_4



# Downsizing
All images in Train folder and moving images to different folders inside of downsized_data. This is done in regular Image_resizing_and_sorting file

# Counting Downsized Images

In [48]:
import os
import os, shutil

total = 0
min_file_count = np.inf
for disease_type in (0, 1, 2, 3, 4):
    path, dirs, files = next(os.walk(downsized_data_path + '/resized_' + str(disease_type)))
    file_count = len(files)
    if file_count < min_file_count:
        min_file_count = file_count
    print('Disease Class of', disease_type, 'has count of images of', file_count)
    total = total + file_count
print('Mininum count by class is', min_file_count)
    
train_file_count = int(min_file_count * .80)
print('Training set will have', train_file_count)
val_file_count = min_file_count - train_file_count
print('Validation & Test set will each have an image count of', int(val_file_count/2))

print('In Total there are', total, 'images in downsized_data folder')

Disease Class of 0 has count of images of 25810
Disease Class of 1 has count of images of 2443
Disease Class of 2 has count of images of 5292
Disease Class of 3 has count of images of 873
Disease Class of 4 has count of images of 708
Mininum count by class is 708
Training set will have 566
Validation & Test set will each have an image count of 71
In Total there are 35126 images in downsized_data folder


# Determining Image Count for Train, Val and Test Folder

In [30]:
total = 0
min_file_count = np.inf
for disease_type in (1, 2, 3, 4):
    path, dirs, files = next(os.walk(downsized_data_path + '/resized_' + str(disease_type)))
    file_count = len(files)
    if file_count < min_file_count:
        min_file_count = file_count
    total = total + file_count

print('In the disease image count is', total)
disease_val_image_count = int(total * .10)
disease_train_image_count = total - disease_val_image_count * 2
print('The disease train image count is',  disease_train_image_count)
print('The disease validation image count is', disease_val_image_count)
print('The disease test image count is', disease_val_image_count )

In the disease image count is 9316
The disease train image count is 7454
The disease validation image count is 931
The disease test image count is 931


## Creating Folders for Training, Validation and Test Set

In [66]:
main_data_folder

'../SML_Project_Data'

In [57]:
create_folder(main_data_folder + '/data', replace = True)
create_folder(main_data_folder + '/data/train', replace = True)
create_folder(main_data_folder + '/data/val', replace = True)
create_folder(main_data_folder + '/data/test', replace = True)

# Create Training Set
for disease in (0, 1):
    for type_path in ('train','val','test'):
        path = main_data_folder + '/data/' + type_path + '/resized_' + str(disease)
        create_folder(path, replace = True)

Attempting to delete folder of ../SML_Project_Data/data
Looking to create folders of ../SML_Project_Data/data

Attempting to delete folder of ../SML_Project_Data/data/train
Folder already does not exist
Looking to create folders of ../SML_Project_Data/data/train

Attempting to delete folder of ../SML_Project_Data/data/val
Folder already does not exist
Looking to create folders of ../SML_Project_Data/data/val

Attempting to delete folder of ../SML_Project_Data/data/test
Folder already does not exist
Looking to create folders of ../SML_Project_Data/data/test

Attempting to delete folder of ../SML_Project_Data/data/train/resized_0
Folder already does not exist
Looking to create folders of ../SML_Project_Data/data/train/resized_0

Attempting to delete folder of ../SML_Project_Data/data/val/resized_0
Folder already does not exist
Looking to create folders of ../SML_Project_Data/data/val/resized_0

Attempting to delete folder of ../SML_Project_Data/data/test/resized_0
Folder already does not

# Copy all files from downsized_data_path to a saved folder

In [68]:
save_downsized_images = False

if save_downsized_images:
    print('Duplicating:' ,downsized_data_path)
    path = downsized_data_path + '_saved'
    print(path)

    copy_tree(src=downsized_data_path, dst=path, verbose=0)

# Move 10% to test and validate, and remaining 80% to Test
## Randomize train_target object, but keep sample individual next to each other

In [19]:
train_target.head()

,image,level,id,side
0,24779_left,2,24779,left
1,24779_right,2,24779,right
2,26490_left,0,26490,left
3,26490_right,0,26490,right
4,40232_left,2,40232,left


In [49]:
import random
import numpy as np

train_target = pd.read_csv(main_data_folder + '/trainLabels.csv', delimiter=',')

train_target['original_level'] = train_target['level']

train_target['level'] = np.where(train_target['level'] == 2, 1, train_target['level'])
train_target['level'] = np.where(train_target['level'] == 3, 1, train_target['level'])
train_target['level'] = np.where(train_target['level'] == 4, 1, train_target['level'])
                                              
#df['First Season'] = np.where(df['First Season'] > 1990, 1, df['First Season'])

train_target[['id','side']] = train_target['image'].str.split(pat = "_", expand=True)

groups = [train_target for _, train_target in train_target.groupby('id')]

random.Random(14).shuffle(groups)

train_target = pd.concat(groups).reset_index(drop=True)
train_target.head()

,image,level,original_level,id,side
0,24779_left,1,2,24779,left
1,24779_right,1,2,24779,right
2,26490_left,0,0,26490,left
3,26490_right,0,0,26490,right
4,40232_left,1,2,40232,left


In [50]:
train_target.drop(['id', 'side'], axis=1, inplace = True)
train_target.head()

,image,level,original_level
0,24779_left,1,2
1,24779_right,1,2
2,26490_left,0,0
3,26490_right,0,0
4,40232_left,1,2


In [51]:
train_target[train_target['level'] == 1].count()

image             9316
level             9316
original_level    9316
dtype: int64

In [52]:
train_target[train_target['level'] == 0].count()

image             25810
level             25810
original_level    25810
dtype: int64

## Move first records in each class to the validate folder

In [58]:
def move_files_val_test(downsized_data_path,
                        main_data_folder,
                        subfolder,
                        val_file_count,
                        move = False,
                       verbose = False):

    counter = 0

    track ={
        0 : 0,
        1 : 0
        }
    missing = 0

    if move == False:
        print('Copying and not moving files')
    
    for index, row in train_target.iterrows():

        # Get training set path of next image
        original_level = row['original_level']
        split_image = row['image'].split('_')
        #print(split_image)
        image_num = split_image[0]
        #print(image_num)
        side = split_image[1]
        #print(side)
        image_name = str(image_num) + '_' + side + '.jpeg'
        #print(image_name)
        downsized_image_path = downsized_data_path + '/resized_' + str(original_level) + '/' + image_name
        #print(downsized_image_path)
        
        # Check that the class of interest isn't full
        if track.get(row['level']) < val_file_count:
            if verbose:
                print('required images is not full')
        
            # Check if file exists
            if path_check(downsized_image_path).is_file():
                #if verbose:
                #    print('file exists of ', train_path)

                # Create new image file path
                output_path = main_data_folder + '/data/' + subfolder + '/resized_' + str(row['level']) + '/' + image_name
                #print('Output Path', output_path)
                
                if move:
                    shutil.move(src=downsized_image_path, dst=output_path)
                else:
                    #print('Copying image from', downsized_image_path, 'to', output_path)
                    shutil.copyfile(src=downsized_image_path, dst=output_path)
                
                counter = counter + 1

                # update counter on folder
                new_value = track.get(row['level']) + 1
                track.update({row['level'] : new_value})
            else:
                #print('Missing image')
                missing = missing + 1

    print('Processed Image Count', counter)

    print(track, 'Missing images are', missing )

for subfolder in ('train','val','test'):
    
    if subfolder == 'train':
        val_file_count=disease_train_image_count
    
    if subfolder == 'val' or subfolder =='test':
        val_file_count=disease_val_image_count
    
    print('Target number is', val_file_count)
    move_files_val_test(downsized_data_path=downsized_data_path,
                            main_data_folder=main_data_folder,
                            subfolder=subfolder,
                            val_file_count=val_file_count,
                            move = True,
                           verbose = False)

Target number is 7454
Processed Image Count 14908
{0: 7454, 1: 7454} Missing images are 0
Target number is 931
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing

Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missin

Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missin

Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missin

Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missin

...

Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missin

Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missin

Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missin

Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missin

Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Missing image
Processed Image Count 1862
{0: 931, 1: 931} Missing images are 16770


# If error, moves saved downsized images back to the downsized images folder.
This will result in all images being in the folder again. This does not fun if error = False

In [75]:
error = False

if error:
    path = downsized_data_path + '_saved'
    copy_tree(src=downsized_data_path + '_saved', dst=downsized_data_path, verbose=0)